# Create Triples

A short guide to adding triples to the Golden decentralized knowledge graph using the godel SDK. Please see the [Golden Protocol FAQ Guide](https://www.notion.so/goldenhq/Golden-Protocol-FAQ-78ae2357b9af44aeaa655cb1b1966ee4) and the [Adding Structured Data Guide](https://www.notion.so/goldenhq/Adding-Structured-Data-Guide-ae657337bf4f4e54ae4402df083c76ac) to learn more about Golden, data submission, and rewards.

Note: Attribution and eligibility for testnet points on triple submissions will be assigned by the earliest timestamped transaction.

## Prerequisite

[Authentication](https://docs.golden.xyz/guides/authentication)

[Godel Authentication](https://docs.golden.xyz/godel-python-sdk/authentication)

This guide requires you install Godel's data-tools.

You can do this with `pip install godel` and comes pre-installed if using the godel docker image.

### 1. Connect to Golden Web3 API

Let's connect the python wrapper to the Golden GraphQL API.

Make sure you ran through the prerequisites for this guide and have learned to authenticate and retrieve your JWT token in Godel.

In [1]:
from godel import GoldenAPI

JWT_TOKEN = ""  # TODO: Enter your JWT token
goldapi = GoldenAPI(jwt_token=JWT_TOKEN)

Test that you can hit the API with `entity_search()` and we'll save the results so we can use the resulting entity as our subject entity later.

In [2]:
import pandas as pd

# Test connection with search
search_results = goldapi.entity_search(name="Barack Obama")
search_results_df = pd.DataFrame(search_results["data"]["entityByName"]["nodes"])
search_results_df

,id,name,description,thumbnail,goldenId,pathname
0,2ebdc8eb-f31b-46d6-a39f-3d544e8b418a,Barack Obama,44th president of the united states of america,https://golden-media.s3.amazonaws.com/topics/2...,207587,/entity/2ebdc8eb-f31b-46d6-a39f-3d544e8b418a
1,bd7f4770-b868-4853-9567-b67d74803e09,Barack Obama,Famous As: 44th President of the United States,https://golden-storage-production.s3.amazonaws...,12275078,/entity/bd7f4770-b868-4853-9567-b67d74803e09
2,a09d4230-f8b1-414f-b0ec-b132d2e71b0c,Barack Obama Sr.,Kenyan economist,None,2066223,/entity/a09d4230-f8b1-414f-b0ec-b132d2e71b0c


### 2. Get Predicates

You can run the code below to get the list of accepted predicates available for the protocol.

In [3]:
import pandas as pd

predicates = {}
for p in goldapi.predicates()["data"]["predicates"]["edges"]:
    p = p["node"]
    predicates[p["name"]] = {"id": p["id"], "objectType": p["objectType"]} 
predicates_df = pd.DataFrame(predicates).transpose()
predicates_df.head()

,id,objectType
Spotify Artist ID,09dec055-52d8-42e6-9fc0-56ecac599a8b,STRING
CEO,0a87e996-34b4-46ba-909a-70ab67b1f811,ENTITY
Email address,0efd0441-1ffc-4e30-8806-e58c434770c8,STRING
YouTube channel,12acb8fe-0573-4ca8-8cc1-180cc6ba3486,ANY_URI
Whitepaper,14fa743c-8161-42e8-a92f-5c29c70e87f8,ANY_URI


### 3. Create Triple

First, view the `CreateStatementInput` object schema.

In [4]:
from godel.schema import CreateStatementInput, StatementInputRecordInput

CreateStatementInput

input CreateStatementInput {
  clientMutationId: String
  subjectId: UUID!
  predicateId: UUID!
  objectValue: String
  objectEntityId: UUID
  citationUrls: [String]
  qualifiers: [QualifierInputRecordInput]
}

Now you can input your triple data. Make sure to check the predicate object type to ensure your object entity ID or object value matches the object type.

#### Triple with Object Value Example

In [5]:
# Triple data
subject_id = search_results_df["id"][0]  # Entity ID
predicate_id = predicates_df["id"]["Website"]  # Website ID
object_value = "https://barackobama.com/"

# Create statement input API
create_statement_input = CreateStatementInput(
    subject_id=subject_id,
    predicate_id=predicate_id,
    object_value=object_value,
    citation_urls=[],
)

### WARNING: Running the code below may charge gas fees and stake testnet points with your wallet. You may lose testnet points by submitting incorrect data.

In [ ]:
data = goldapi.create_statement(create_statement_input=create_statement_input)
data

#### Triple with Object Entity ID Example

In [ ]:
# Triple data
subject_id = "6a20066c-0909-4cd6-81ab-c12245a285e0"  # "John Doe" Entity ID
predicate_id = predicates["CEO of"]["id"]  # "CEO of" Predicate ID
object_entity_id = "36b56cea-db86-482f-8b63-5b4e75b3b968"  # "Huel" Entity ID

# Create statement input API
create_statement_input = CreateStatementInput(
    subject_id=subject_id,
    predicate_id=predicate_id,
    object_entity_id=object_entity_id,
    citation_urls=["https://golden.com/wiki/johndoe"],
)

### WARNING: Running the code below may charge gas fees and stake testnet points with your wallet. You may lose testnet points by submitting incorrect data.

In [ ]:
data = goldapi.create_statement(create_statement_input=create_statement_input)
data